In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
from aicsimageio import AICSImage
from aicsimageio.readers.ome_tiff_reader import OmeTiffReader
import matplotlib.pyplot as plt
%matplotlib notebook
%matplotlib inline
import seaborn as sns
import textwrap

import sys
src_path = str(Path.cwd().parent)
if src_path not in sys.path:
    sys.path.append(src_path)
from src.d00_utils.dirnames import excl_analysis_dirname
from src.d00_utils import utilities as utils
from src.d00_utils import dirnames as dn

In [ ]:
# Channels
caax_live_ch = 0
caax_fix_ch = 2
cell_ch = 1
MBP_ch = 3
excl_live_ch = 4
excl_fix_ch = 5

ch_labels = {caax_live_ch: 'CAAX (live)',
             caax_fix_ch: 'CAAX (fix)', 
             cell_ch: 'cell', MBP_ch: 'MBP', 
             excl_live_ch: 'CAAX-excluded (live)', 
             excl_fix_ch: 'CAAX-excluded (fix)'}

ch_save_abbr = {caax_live_ch: 'live_caax', 
                caax_fix_ch: 'fix_caax', 
                cell_ch: 'memb', 
                MBP_ch: 'MBP', 
                excl_live_ch: "live_excl", 
                excl_fix_ch: "fix_excl"}

In [ ]:
img_dirpath = '/Users/kwu2/Library/CloudStorage/GoogleDrive-kwu2@stanford.edu/My Drive/Lab/CryoEM_live_imaging/Experiments/CE012/CZI_to_process/div7_posttimelapse_and_div7_fixed_stained_aligned/img_processing/analyses/ratiometric/MBP_over_fix_caax/ratiometric_arrays'
#img_ch = caax_fix_ch # Only if relevant
y_val = 'caax intensity'
yval_save = 'caax_int'

In [ ]:
binary_dirpath = '/Users/kwu2/Library/CloudStorage/GoogleDrive-kwu2@stanford.edu/My Drive/Lab/CryoEM_live_imaging/Experiments/CE012/CZI_to_process/div7_posttimelapse_and_div7_fixed_stained_aligned/img_processing/analyses/exclusion_analysis/perc_area/binary_regions_memb-mask'

In [ ]:
# Convert strings to Path objects
img_dirpath = Path(img_dirpath)
binary_dirpath = Path(binary_dirpath)

# Get output directory path
proc_dirpath = utils.get_proc_dirpath(img_dirpath)
output_dirpath = proc_dirpath / dn.analyses_dirname / dn.int_dirname / yval_save
output_dirpath.mkdir(parents=True, exist_ok=True)

# Get all files matching the extensions listed
exts = ('.npy', 'ome.tif')
files = []
for ext in exts:         
    files.extend(img_dirpath.glob(f'*{ext}'))
imgpaths = [path for path in files]

overall_data_df = pd.DataFrame()

for imgpath in imgpaths:
    base_imgname = imgpath.name.split('.')[0]
    
    # Check whether there is a matching binary file
    binpath = binary_dirpath / (base_imgname + '.ome.tif')
    if binpath.exists():
        
        # Open image (depends on file format)
        if imgpath.suffix == '.npy':
            img = np.load(imgpath)

        else:
            img_file = AICSImage(imgpath, reader=OmeTiffReader)
            img = img_file.data[:, img_ch, :, :, :].squeeze()
        
        # Open binary file
        binary_file = AICSImage(binpath, reader=OmeTiffReader)
        binary = (binary_file.data).astype(bool)
        
        # Input data
        idv_data = {'initial image directory': imgpath.parent.name}
        info_df = utils.extract_img_info(imgpath.name)
        
        # Measure y-values for each binary region
        size_c = binary.shape[1]
        for binreg in range(size_c):
            img_reg = img * binary[:, binreg, :, :, :].squeeze()
            #avgval_reg = np.nanmean(img_reg[img_reg > 0])
            #idv_data.update({f'mean {y_val} in {ch_labels[binreg]} region': avgval_reg})
            perc_over1 = np.count_nonzero(img_reg[img_reg > 1]) / np.count_nonzero(img_reg[img_reg > 0])
            idv_data.update({f'{y_val} in {ch_labels[binreg]} region': perc_over1})
            
            plt.imshow(img_reg)
            plt.show()

        idv_data_df = pd.DataFrame(idv_data, index=[0])
        idv_data_df = pd.concat([info_df, idv_data_df], axis=1)
        overall_data_df = pd.concat([overall_data_df, idv_data_df], ignore_index=True)
        
#overall_data_df.to_csv(output_dirpath / f'mean_{yval_save}.csv')
overall_data_df.to_csv(output_dirpath / f'perc_over1.csv')
print('done')
        

In [ ]:
def wrap_xticklabels(ax, width=15, break_long_words=False):
    xticklabels = []
    for xticklab in ax.get_xticklabels():
        text = xticklab.get_text()
        xticklabels.append(textwrap.fill(text, width=width, break_long_words=break_long_words))
    ax.set_xticklabels(xticklabels)

In [ ]:
overall_data_df = pd.read_csv('/Users/kwu2/Library/CloudStorage/GoogleDrive-kwu2@stanford.edu/My Drive/Lab/CryoEM_live_imaging/Experiments/CE012/CZI_to_process/div7_posttimelapse_and_div7_fixed_stained_aligned/img_processing/analyses/intensities/caax_int/mean_caax_int.csv')

In [ ]:
binregs_plot = [caax_live_ch, caax_fix_ch, excl_live_ch, excl_fix_ch, cell_ch, MBP_ch]

# Reshape dataframe
img_info_cols = overall_data_df.columns[:7]
x_label = 'binarized region'
y_label = f'mean {y_val}'
overall_data_dfm = overall_data_df.melt(img_info_cols, var_name=x_label, value_name=y_label)

# Filter reshaped dataframe
filter_plot = []
for binreg in binregs_plot:
    filter_plot.append(f'mean {y_val} in {ch_labels[binreg]} region')
    #filter_plot.append(f'{y_val} in {ch_labels[binreg]} region')
print(filter_plot)

filtered_dfm = overall_data_dfm.loc[overall_data_dfm[x_label].isin(filter_plot)]

# Plot data
fig = plt.figure(figsize=(1.5 * len(binregs_plot), 5))
ax = sns.swarmplot(filtered_dfm, x=x_label, y=y_label, hue='UID', order=filter_plot)
ax.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

# Set x-tick label names and wrap text as necessary
ax.set_xticks(range(len(binregs_plot)), labels=[ch_labels[binreg] for binreg in binregs_plot])
wrap_xticklabels(ax, width=15)

# Set y-axis limits
ymax = np.round(np.max(filtered_dfm[y_label]) * 1.2, 1)
ax.set_ylim(0, ymax)

binreg_str = ''
for binreg in binregs_plot:
    binreg_str = binreg_str + '_' + ch_save_abbr[binreg]
    

fig.savefig(output_dirpath / f'{yval_save}{binreg_str}.pdf', bbox_inches='tight')